In [16]:
!pip install nltk
!pip install xgboost

In [17]:
import pandas as pd

data_size = '10p'
# data_size = 'full'
version = 'v0'

data = pd.read_csv('../data/neiss_10p_sample.csv') if data_size == '10p' else pd.read_csv('../data/consolidated_cleaned_neiss_2014_2023.csv')
embedding = pd.read_csv(f'../data/gist_embedding_{data_size}_{version}.csv')

data = data[['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1', 'Disposition']]
data = data.merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)

# data.to_csv('../data/consolidated_cleaned_neiss_2014_2023_with_embedding.csv', index=False)
# data = pd.read_csv('../data/consolidated_cleaned_neiss_2014_2023_with_embedding.csv')

data.head(5)
data.columns

Index(['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1',
       'Disposition', '0', '1', '2',
       ...
       '374', '375', '376', '377', '378', '379', '380', '381', '382', '383'],
      dtype='object', length=391)

In [18]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [19]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts() 

Disposition_recode_2
0    295967
1     33465
Name: count, dtype: int64

In [20]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [21]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
# data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [22]:
total_rows, n_columns = data.shape

test_size = int(total_rows * 0.2)
train_size = total_rows - test_size

print(f"Original dataset shape: ({total_rows}, {n_columns})")
print(f"Train set size: {train_size} rows")
print(f"Test set size: {test_size} rows")
data = data.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle the data

data_train = data.tail(train_size).reset_index(drop=True)
data_test = data.head(test_size).reset_index(drop=True)
# data_train=data.tail(191347).reset_index(drop=True)
# data_test=data.head(21000).reset_index(drop=True)

data_train.columns

Original dataset shape: (329432, 394)
Train set size: 263546 rows
Test set size: 65886 rows


Index(['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1',
       'Disposition', '0', '1', '2',
       ...
       '377', '378', '379', '380', '381', '382', '383', 'Disposition_recode',
       'Disposition_recode_2', 'body_string'],
      dtype='object', length=394)

In [23]:
drop_list=[
 'CPSC_Case_Number',
 'Disposition',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
]

drop_list_test=[
 'CPSC_Case_Number',
 'Disposition',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
]

X = data_train.drop(drop_list, axis=1) 
y = data_train['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [ ]:
import pickle
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Separate features (X) and target variable (y)
X = data_train.drop(drop_list, axis=1) 
y = data_train['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_test.drop(drop_list_test, axis=1)
y_test=data_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open(f'X_scaler_{data_size}_{version}.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic'), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision',n_jobs=-1)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open(f'xgboost_tfidf_LLM_{data_size}_{version}.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...


In [ ]:
X_test_final.head(5)

,Age,Sex,Body_Part,Location,Product_1,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,0.565123,-0.927244,-1.350384,-0.758252,-0.224069,0.660472,0.082750,0.142342,0.364615,-0.042726,...,-1.147385,0.598953,0.655564,-0.950621,0.203397,-1.489740,1.143341,-0.310971,-0.115412,0.174948
1,-0.631854,1.078465,0.453370,-0.758252,-0.192996,-1.061419,-1.000939,-0.596747,-0.602457,-0.768384,...,-1.465967,0.541281,0.072080,-0.046963,-0.418185,-0.256626,-1.319661,-1.320048,0.929812,-0.745724
2,-0.589855,-0.927244,-1.436277,1.728689,2.144190,0.943535,-0.346995,-0.832362,-0.972159,0.246567,...,1.310636,0.362386,-0.022807,1.137247,0.322863,-0.239249,-0.113168,-0.616188,2.330140,-0.222589
3,-0.085865,-0.927244,-1.436277,-0.447384,-0.240346,-0.091793,-0.546850,1.017554,-2.033134,1.568280,...,-0.608941,-0.346526,0.693909,1.057529,0.892148,-0.614567,-1.806765,-0.923758,-0.204613,-0.260864
4,-0.778851,-0.927244,-1.350384,0.485219,0.853892,-0.553534,0.415949,0.828780,0.374225,-0.783442,...,1.098096,0.280247,0.499938,-0.569146,-0.051937,-0.713674,-0.314435,1.536370,-1.029749,-0.002860


In [ ]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

array([-1.47922387, -1.43627734, -1.39333081, -1.35038429, -1.30743776,
       -1.26449123, -1.2215447 , -1.17859818, -1.13565165,  0.45336985,
        0.49631638,  0.53926291,  0.62515596,  0.66810249,  0.71104902,
        0.75399554,  0.79694207,  1.01167471,  1.05462123,  1.18346082,
        1.22640734,  1.26935387])

In [ ]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

0.27455264 0.8904435
0.77868474 0.98962057
0.656549 0.98519975
0.41402954 0.935338
0.4990844 0.9728259
0.5531249 0.9868457
0.7142164 0.98793447
0.58260804 0.96586573
0.28285968 0.91216266
0.34838432 0.94888216
